In [4]:
import os
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession 
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

conf = (
    SparkConf()
    .setAppName("knmi daily weather to bronze")
    .set("spark.hadoop.fs.s3a.endpoint", "http://192.168.86.192:9000")
    .set("spark.hadoop.fs.s3a.access.key", os.getenv('MINIO_ROOT_USER'))
    .set("spark.hadoop.fs.s3a.secret.key", os.getenv('MINIO_ROOT_PASSWORD'))
    .set("spark.hadoop.fs.s3a.path.style.access", True)
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .set("spark.driver.memory", "8g")
    .set("spark.executor.memory", "8g")
    .set("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") 
    .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") 
    .set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") 
    .set("spark.sql.catalog.spark_catalog.type", "hadoop") 
    .set("spark.sql.catalog.spark_catalog.warehouse", "s3a://warehouse") 
    .set('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
)
sc = SparkContext(conf=conf).getOrCreate()
sc.setLogLevel("ERROR")
spark = SparkSession(sc).builder.getOrCreate()


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=knmi daily weather to bronze, master=local[*]) created by __init__ at /tmp/ipykernel_44/826370988.py:23 

In [11]:
df = spark.read.format('delta').load('s3a://bronze-knmi/daggegevens')

In [6]:
#df.write.option('path', 's3a://test-bucket/testtable').saveAsTable('test_table2')


In [11]:
df.select(df.weather_station_code, df.date).show()

+--------------------+----------+
|weather_station_code|      date|
+--------------------+----------+
|                 286|1999-11-18|
|                 331|1999-11-18|
|                 235|1999-11-18|
|                 240|1999-11-18|
|                 380|1999-11-18|
|                 210|1999-11-18|
|                 277|1999-11-18|
|                 275|1999-11-18|
|                 323|1999-11-18|
|                 344|1999-11-18|
|                 285|1999-11-18|
|                 330|1999-11-18|
|                 348|1999-11-18|
|                 265|1999-11-18|
|                 251|1999-11-18|
|                 283|1999-11-18|
|                 267|1999-11-18|
|                 225|1999-11-18|
|                 324|1999-11-18|
|                 278|1999-11-18|
+--------------------+----------+
only showing top 20 rows



NameError: name 'spark' is not defined

In [13]:
spark.sql('''
CREATE DATABASE jdp
''')

DataFrame[]

In [14]:
spark.sql('''USE jdp''')

DataFrame[]

In [23]:
spark.sql('''
CREATE TABLE IF NOT EXISTS test2 (id int, name string)  USING delta LOCATION 's3a://warehouse/'
''')

DataFrame[]

In [24]:
spark.sql('''INSERT INTO test VALUES (1, 'my first insert')''')

DataFrame[]

In [17]:
spark.sql('''SELECT * FROM spark_catalog.default.test''').show()

+---+---------------+
| id|           name|
+---+---------------+
|  1|my first insert|
+---+---------------+



In [27]:
spark.sql('''DESCRIBE HISTORY test2''').show()

+-------+-------------------+------+--------+------------+--------------------+----+--------+---------+-----------+--------------+-------------+----------------+------------+
|version|          timestamp|userId|userName|   operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics|userMetadata|
+-------+-------------------+------+--------+------------+--------------------+----+--------+---------+-----------+--------------+-------------+----------------+------------+
|      0|2022-12-20 20:53:43|  null|    null|CREATE TABLE|{isManaged -> fal...|null|    null|     null|       null|          null|         true|              {}|        null|
+-------+-------------------+------+--------+------------+--------------------+----+--------+---------+-----------+--------------+-------------+----------------+------------+



In [20]:
spark.sql.catalog.listDatabases()

AttributeError: 'function' object has no attribute 'catalog'

In [30]:
df = spark.read.load('s3a://bronze-knmi/daggegevens')

In [31]:
df.createOrReplaceTempView('knmi')

In [35]:
spark.sql('''select count(DISTINCT YYYYMMDD) from knmi''').show()

+------------------------+
|count(DISTINCT YYYYMMDD)|
+------------------------+
|                   44548|
+------------------------+



In [37]:
spark.sql('''create table if not exists bronze_daggegevens USING delta LOCATION 's3a://bronze-knmi/daggegevens'  ''')

DataFrame[]

In [38]:
spark.sql('''DESCRIBE HISTORY bronze_daggegevens''').show()

+-------+-------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+
|version|          timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|
+-------+-------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+
|      1|2022-12-20 21:22:19|  null|    null|    WRITE|{mode -> Overwrit...|null|    null|     null|          0|          null|        false|{numFiles -> 4, n...|        null|
|      0|2021-08-31 21:07:13|  null|    null|    WRITE|{mode -> Overwrit...|null|    null|     null|       null|          null|        false|{numFiles -> 16, ...|        null|
+-------+-------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------

In [101]:
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, "s3a://bronze-knmi/daggegevens")
deltaTable.history().show()

+-------+-------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+
|version|          timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|
+-------+-------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+
|      9|2022-12-22 21:26:53|  null|    null|    MERGE|{predicate -> ((k...|null|    null|     null|          8|          null|        false|{numTargetRowsCop...|        null|
|      8|2022-12-21 21:06:10|  null|    null|    MERGE|{predicate -> ((k...|null|    null|     null|          7|          null|        false|{numTargetRowsCop...|        null|
|      7|2022-12-20 22:05:04|  null|    null|    MERGE|{predicate -> ((k...|null|    null|     null|          6|        

In [52]:
print(spark.read.format("delta").option("versionAsOf", 0).load("s3a://bronze-knmi/daggegevens").count())

849211


In [53]:
print(spark.read.format("delta").option("versionAsOf", 1).load("s3a://bronze-knmi/daggegevens").count())

23500


In [57]:
print(spark.read.format("delta").option("versionAsOf", 2).load("s3a://bronze-knmi/daggegevens").count())

849211


In [71]:
print(spark.read.format("delta").option("versionAsOf", 5).load("s3a://bronze-knmi/daggegevens").count())

895177


In [78]:
df = spark.read.format("delta").load("s3a://bronze-knmi/daggegevens")


In [79]:
df.createOrReplaceTempView('dag')

In [80]:
spark.sql('''

select STN, YYYYMMDD, load_datetime from dag where YYYYMMDD = '20220101' order by STN

''').show()

+---+--------+-------------+
|STN|YYYYMMDD|load_datetime|
+---+--------+-------------+
+---+--------+-------------+



In [76]:
df = spark.read.format("delta").option("versionAsOf", 0).load("s3a://bronze-knmi/daggegevens")

In [2]:
df = spark.read.format("delta").load("s3a://bronze-knmi/daggegevens")

In [3]:
from pyspark.sql.functions import desc
df.select('load_datetime', 'YYYYMMDD').orderBy(desc('load_datetime')).show()

+--------------------+--------+
|       load_datetime|YYYYMMDD|
+--------------------+--------+
|2022-12-22 21:26:...|20221221|
|2022-12-22 21:26:...|20221221|
|2022-12-22 21:26:...|20221221|
|2022-12-22 21:26:...|20221221|
|2022-12-22 21:26:...|20221221|
|2022-12-22 21:26:...|20221221|
|2022-12-22 21:26:...|20221221|
|2022-12-22 21:26:...|20221221|
|2022-12-22 21:26:...|20221221|
|2022-12-22 21:26:...|20221221|
|2022-12-22 21:26:...|20221221|
|2022-12-22 21:26:...|20221221|
|2022-12-22 21:26:...|20221221|
|2022-12-22 21:26:...|20221221|
|2022-12-22 21:26:...|20221221|
|2022-12-22 21:26:...|20221221|
|2022-12-22 21:26:...|20221221|
|2022-12-22 21:26:...|20221221|
|2022-12-22 21:26:...|20221221|
|2022-12-22 21:26:...|20221221|
+--------------------+--------+
only showing top 20 rows



In [105]:
rdw_raw_df = spark.read.csv('s3a://test-bucket/rdw/', header=True)

In [108]:
rdw_raw_df.groupBy('kenteken').count().filter('count > 1').show()

+--------+-----+
|kenteken|count|
+--------+-----+
|  00LHG1|    2|
|  00TSB9|    2|
|  00VXS7|    2|
|  01JDPS|    2|
|  01LTRH|    2|
|  01NTN4|    2|
|  01SFB4|    2|
|  00LDF3|    2|
|  01JPH4|    2|
|  02LLHX|    2|
|  02NXK2|    2|
|  02XFT6|    2|
|  02XKK2|    2|
|  02XZPN|    2|
|  02YB99|    2|
|  03GHTL|    2|
|  03JTG1|    2|
|  03SXB1|    2|
|  03TJH9|    2|
|  03XPH8|    2|
+--------+-----+
only showing top 20 rows



In [120]:
rdw_df =  rdw_raw_df.select('kenteken', col('Brandstof volgnummer').alias('brandstof_volgnummer'), col('Brandstof omschrijving').alias('brandstof_omschrijving'), col('Klasse hybride elektrisch voertuig').alias('klasse_hybride'))

In [123]:
rdw_df.coalesce(1).write.format('parquet').mode('overwrite').save('s3a://test-bucket/rdw_brandstof/')

In [119]:
from pyspark.sql.functions import col

In [124]:
rdw_df.createOrReplaceTempView('rdw')

In [127]:
spark.sql('''

select kenteken, count(*), collect_list(brandstof_omschrijving)
from rdw
group by kenteken
having count(*) > 1

''').show()

+--------+--------+------------------------------------+
|kenteken|count(1)|collect_list(brandstof_omschrijving)|
+--------+--------+------------------------------------+
|  00LDF3|       2|                [Benzine, Elektri...|
|  00LHG1|       2|                [Benzine, Elektri...|
|  00TSB9|       2|                [Benzine, Elektri...|
|  00VXS7|       2|                      [LPG, Benzine]|
|  01JDPS|       2|                      [Benzine, LPG]|
|  01JPH4|       2|                [Benzine, Elektri...|
|  01LTRH|       2|                      [Benzine, LPG]|
|  01NTN4|       2|                      [LPG, Benzine]|
|  01SFB4|       2|                [Benzine, Elektri...|
|  02LLHX|       2|                      [Benzine, LPG]|
|  02NXK2|       2|                [Benzine, Elektri...|
|  02XFT6|       2|                [Benzine, Elektri...|
|  02XKK2|       2|                [Benzine, Elektri...|
|  02XZPN|       2|                      [LPG, Benzine]|
|  02YB99|       2|            

In [16]:
df.createOrReplaceTempView('dag')

In [14]:
spark.sql('''
SELECT distinct YYYYMMDD from dag order by YYYYMMDD desc
''').show()

+--------+
|YYYYMMDD|
+--------+
|20230128|
|20230127|
|20230126|
|20230125|
|20230124|
|20230123|
|20230122|
|20230121|
|20230120|
|20230119|
|20230118|
|20230117|
|20230116|
|20230115|
|20230114|
|20230113|
|20221221|
|20221220|
|20221219|
|20221218|
+--------+
only showing top 20 rows



In [15]:
df = spark.read.format('delta').load('s3a://silver-knmi/daggegevens')

In [18]:
spark.sql('''
SELECT date, count(*) from dag group by date order by 1 desc
''').show()

+----------+--------+
|      date|count(1)|
+----------+--------+
|2023-01-29|      47|
|2023-01-28|      47|
|2023-01-27|      47|
|2023-01-26|      47|
|2023-01-25|      48|
|2023-01-24|      48|
|2023-01-23|      48|
|2023-01-22|      47|
|2023-01-21|      58|
|2023-01-20|      47|
|2023-01-19|      47|
|2023-01-18|      48|
|2023-01-17|      51|
|2023-01-16|      47|
|2023-01-15|      47|
|2023-01-14|      61|
|2023-01-13|      47|
|2023-01-12|      47|
|2023-01-11|      47|
|2023-01-10|      47|
+----------+--------+
only showing top 20 rows



In [30]:
df = spark.read.format('delta').load("s3a://bronze-knmi/uurgegevens").filter('YYYYMMDD is not null')

In [36]:
df.drop('load_datetime').repartition(4).write.mode('overwrite').format('parquet').save("s3a://fontys/knmi/hourly_weather")

In [32]:
df.createOrReplaceTempView('uur')

In [33]:
spark.sql('''
select STN, YYYYMMDD, H, count(*)
from uur
group by STN, YYYYMMDD, H
having count(*) > 1
''').show()

+---+--------+---+--------+
|STN|YYYYMMDD|  H|count(1)|
+---+--------+---+--------+
+---+--------+---+--------+



In [37]:
df = spark.read.format('parquet').load("s3a://fontys/knmi/hourly_weather")

In [38]:
df.createOrReplaceTempView('hourly')

In [39]:
spark.sql('''
select left(YYYYMMDD, 4), count(*)
from hourly
group by left(YYYYMMDD, 4)
order by 1 desc
''').show()

+-----------------+--------+
|left(YYYYMMDD, 4)|count(1)|
+-----------------+--------+
|             2023|   47472|
|             2022|  402960|
|             2021|  402960|
|             2020|  412848|
|             2019|  411720|
|             2018|  411720|
|             2017|  403392|
|             2016|  412440|
|             2015|  420480|
|             2014|  416544|
|             2013|  420480|
|             2012|  421632|
|             2011|  420480|
|             2010|  411720|
|             2009|  411720|
|             2008|  418200|
|             2007|  415248|
|             2006|  419736|
|             2005|  411696|
|             2004|  412848|
+-----------------+--------+
only showing top 20 rows



In [40]:
df = spark.read.format('delta').load('s3a://bronze-knmi/weather_stations')

In [44]:
df.drop('load_datetime').dropDuplicates().repartition(1).write.mode('overwrite').format('parquet').save('s3a://fontys/knmi/weather_stations')